In [ ]:
%%capture
from jupyter_dash import JupyterDash
import pandas as pd
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

app = JupyterDash(__name__)

poverty = pd.read_csv('../../data/poverty.csv', low_memory=False)
gini = 'GINI index (World Bank estimate)'
gini_df = poverty[poverty[gini].notna()]

app.layout = html.Div([
    html.Br(),
    html.H2('Gini Index - World Bank Data', style={'textAlign': 'center'}),
    html.Br(),
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(id='gini_year_dropdown',
                         options=[{'label': y, 'value': y}
                                  for y in gini_df['year'].drop_duplicates().sort_values()]),
            html.Br(),
            dcc.Graph(id='gini_year_barchart')
        ]),
        dbc.Col([
            dcc.Dropdown(id='gini_country_dropdown',
                         multi=True,
                         options=[{'label': country, 'value': country}
                                  for country in gini_df['Country Name'].unique()]),
            html.Br(),
            dcc.Graph(id='gini_country_barchart')
        ]),
    ])
])

@app.callback(Output('gini_year_barchart', 'figure'),
              Input('gini_year_dropdown', 'value'))
def plot_gini_year_barchart(year):
    if not year:
        raise PreventUpdate
    df = gini_df[gini_df['year'].eq(year)].sort_values(gini).dropna(subset=[gini])
    n_countries = len(df['Country Name'])
    fig = px.bar(df,
                 x=gini,
                 y='Country Name', 
                 orientation='h',
                 height=200 + (n_countries*20), 
                 title=gini + ' ' + str(year))
    return fig


@app.callback(Output('gini_country_barchart', 'figure'),
              Input('gini_country_dropdown', 'value'))
def plot_gini_country_barchart(countries):
    if not countries:
        raise PreventUpdate
    df = gini_df[gini_df['Country Name'].isin(countries)].dropna(subset=[gini])
    fig = px.bar(df,
                 x='year',
                 y=gini, 
                 height=100 + (250*len(countries)),
                 facet_row='Country Name',
                 labels={gini: 'Gini Index'},
                 color='Country Name',
                 title=''.join([gini, '<br>', '<b>',
                                ', '.join(countries), '</b>']))
    return fig

if __name__ == '__main__':
    app.run_server(host='0.0.0.0', mode='inline',port=3000)

In [1]:
! echo 'Your app is running on : '$EDUCATIVE_LIVE_VM_URL':3000'

Your app is running on : :3000
